In [146]:
import numpy
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import choice
import pandas as pd


def get_page(page, user_agent, proxy):
    url = "https://rateyourmusic.com/customchart?page={}".format(page)
    html_page = requests.get(url, headers=user_agent, proxies=proxy)
    file = open("./Docs/{}.html".format(page), "w")
    file.write(str(html_page.text))
    file.close()
    return 0


def downloader():
    file_of_users_agents = open("user_agents.txt", "r")
    users_agents = file_of_users_agents.read().split('\n')
    file_of_proxies = open("proxies.txt", "r")
    proxies = file_of_proxies.read().split('\n')
    file_of_proxies.close()
    file_of_users_agents.close()
    for j in range(1,5):
        for i in range(j,26,4):
            proxy = {'http': 'http://' + choice(proxies)}
            user_agent = {'User-Agent': choice(users_agents)}
            get_page(i, user_agent, proxy)
            sleep(90)

def parse():
    results = []
    for i in range(1, 26):
        text = open("./Docs/{}.html".format(i), "r")
        soup = BeautifulSoup(text, "lxml")
        for elem in soup.find_all('tr'):
            try:
                artist_and_album = elem.find('div', class_='chart_main')
                artist_name = artist_and_album.find('span', class_='chart_detail_line1').find('a').text
                album_name = artist_and_album.find('div', class_='chart_detail_line2').find('a').text
                ratings = elem.find('div', class_='chart_stats')
                RYM_rating= float(ratings.find('b').text)
                rating = int(ratings.find('b').find_next_sibling('b').text.replace(',', ''))
                results.append({
                'artist_name': artist_name,
                'album_name': album_name,
                'RYM_rating': RYM_rating,
                'rating': rating,
            })
            except:
                continue
    df = pd.DataFrame(results)
    df.to_csv('results.csv')
    return df


def analysis(df):
    print("This is most popular artist : {} - {}".format(df['artist_name'].value_counts().index[0], df['artist_name'].value_counts().values[0]))
    top10_artists = df
    top10_artists = top10_artists.groupby('artist_name').aggregate(sum)
    top10_artists = top10_artists.rating.sort_values(ascending=False)[:10]
    print("\n")
    print("This is TOP10 of the most popular artists:\n{}".format(top10_artists))
    top10_rate_artist = df.groupby('artist_name').aggregate(sum)
    all_=df.groupby('artist_name').count()
    top10_rate_artist['count']=all_['RYM_rating']
    top10_rate_artist['RYM_rating']=round(top10_rate_artist['RYM_rating']/top10_rate_artist['count'],2)
    top10_rate_artist = top10_rate_artist.RYM_rating.sort_values(ascending=False)[:10]
    print("\n")
    print("This is TOP10 of the most rate artists:\n{}".format(top10_rate_artist))


#downloader()
analysis(parse())


This is most popular artist : Miles Davis - 11


This is TOP10 of the most popular artists:
artist_name
Radiohead                 188430
The Beatles               182181
Pink Floyd                155550
David Bowie               114193
Led Zeppelin              104991
Bob Dylan                  98562
Black Sabbath              86508
The Velvet Underground     67800
Metallica                  67210
Miles Davis                65662
Name: rating, dtype: int64


This is TOP10 of the most rate artists:
artist_name
Mingus                         4.35
The Beach Boys                 4.21
Nas                            4.18
Television                     4.18
Wu-Tang Clan                   4.18
Madvillain                     4.13
Godspeed You Black Emperor!    4.13
Genius/GZA                     4.12
The Jimi Hendrix Experience    4.12
Joy Division                   4.12
Name: RYM_rating, dtype: float64
